# Análisis Exploratorio de Datos (EDA): 

## Objetivo
Este notebook contiene el análisis exploratorio completo del dataset, siguiendo una estructura organizada y metódica para entender los datos, limpiarlos y prepararlos para modelado.

## Contenido
1. Configuración Inicial y librearías
2. Carga y Vista General de los Datos
3. Limpieza y Preprocesamiento
4. Análisis Exploratorio de los Datos
5. Análisis estadístico
6. Análisis Univariante
7. Análisis Bivariante
8. Conclusiones

Esto es una prueba 

In [3]:
import pandas as pd 
import os

In [2]:
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', None) 

# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None)     # Show all rows
# pd.set_option('display.max_colwidth', None) # Show entire content of each column
pd.set_option('display.width', None)        # No limit on display width
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

In [4]:
# Setting paths
current_dir = os.getcwd()  # Use os.getcwd() to get the current working directory
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, "data")
file_path = os.path.join(data_dir, "")

cars_csv_path = os.path.join(data_dir, "train.csv")

#----------------------------------------------------------------------------------------------------
# Creading dataframes
df_car = pd.read_csv(cars_csv_path)

In [5]:
df_car.sample(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
119982,119982,Audi,A7 55 Premium,2022,12000,Hybrid,335.0HP 3.0L V6 Cylinder Engine Gasoline/Mild ...,A/T,White,Black,None reported,Yes,75000
84843,84843,Ford,F-250 XL,2016,29719,E85 Flex Fuel,385.0HP 6.2L 8 Cylinder Engine Flex Fuel Capab...,A/T,White,Gray,None reported,Yes,31950
94580,94580,Land,Rover Range Rover Sport HSE,2016,87500,Gasoline,340.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,28000
58,58,Land,Rover Range Rover Sport HSE,2008,122000,Gasoline,300.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes,8000
5889,5889,BMW,M235 i,2013,82000,Gasoline,320.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Red,None reported,Yes,14999
